In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
from keras.models import *
from keras.layers import *
from keras.datasets import mnist

In [ ]:
#상수선언
OUT_DIR = './CNN_out'
img_shape = (28, 28, 1)
epochs = 100000
batch_size = 128
noise = 100
sample_interval = 100       # 100마다 샘플뽑아서 저장

In [ ]:
#
(x_train, _), (_, _) = mnist.load_data()     # _ == ytrain, xtest, ytest 필요없어서 _로 표시
print(x_train.shape)

x_train = x_train / 127.5 - 1   # -1 에서 1 사이의 값이 나옴
x_train = np.expand_dims(x_train, axis=3)   # expand_dims : 익스펜드 디멘션 -> 차원을 하나 늘려라
print(x_train.shape)

(60000, 28, 28)
(60000, 28, 28, 1)


In [ ]:
# 강사님 제너레이터
generator = Sequential()
generator.add(Dense(256*7*7, input_dim=noise))
generator.add(Reshape((7, 7, 256)))
generator.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))
generator.add(BatchNormalization())
generator.add(LeakyReLU(alpha=0.01))
generator.add(Conv2DTranspose(64, kernel_size=3, strides=1, padding='same'))
generator.add(BatchNormalization())
generator.add(LeakyReLU(alpha=0.01))
generator.add(Conv2DTranspose(1, kernel_size=3, strides=2, padding='same'))
generator.add(Activation('tanh'))
generator.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 12544)             1266944   
                                                                 
 reshape_3 (Reshape)         (None, 7, 7, 256)         0         
                                                                 
 conv2d_transpose_4 (Conv2DT  (None, 14, 14, 128)      295040    
 ranspose)                                                       
                                                                 
 batch_normalization_2 (Batc  (None, 14, 14, 128)      512       
 hNormalization)                                                 
                                                                 
 leaky_re_lu_6 (LeakyReLU)   (None, 14, 14, 128)       0         
                                                                 
 conv2d_transpose_5 (Conv2DT  (None, 14, 14, 64)      

In [ ]:
# 제너레이터
# generator = Sequential()
# generator.add(Dense(256*7*7, input_dim=noise))  # 레이어 100개      # activation='leaky_relu'
# generator.add(Reshape((7, 7, 256)))             # 콘브했다가 맥스풀 # 튜플 형태로 주기 위해 괄호 한번 더
# generator.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))          # 콘브2d와 어셉? 한번에
# generator.add(BatchNormalization())
# generator.add(LeakyReLU(alpha=0.01))
# generator.add(Reshape((7, 7, 256)))             # 바로 위 3줄 복붙
# generator.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))          # 콘브2d와 어셉? 한번에
# generator.add(BatchNormalization())
# generator.add(LeakyReLU(alpha=0.01)) 
# generator.add(Conv2DTranspose(1, kernel_size=3, strides=2, padding='same')
# generator.add(Activation('tanh'))

# generator.add(LeakyReLU(alpha=0.01)) # 진한 부분만 보고 처리할라고 리키렐루에 적용
# generator.add(Dense(784, activation='tanh'))  # 하이퍼볼릭 탄젠트: 마이너스 값이라서 tanh을 적용
# generator.add(Reshape(img_shape))
# generator.summary()

In [ ]:
discriminator = Sequential()
discriminator.add(Conv2D(32, kernel_size=3, strides=2, padding='same', input_shape=img_shape))
discriminator.add(LeakyReLU(alpha=0.01))
discriminator.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
discriminator.add(LeakyReLU(alpha=0.01))
discriminator.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
discriminator.add(LeakyReLU(alpha=0.01))
discriminator.add(Flatten())
discriminator.add(Dense(1, activation='sigmoid'))
discriminator.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 14, 14, 32)        320       
                                                                 
 leaky_re_lu_8 (LeakyReLU)   (None, 14, 14, 32)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 7, 7, 64)          18496     
                                                                 
 leaky_re_lu_9 (LeakyReLU)   (None, 7, 7, 64)          0         
                                                                 
 conv2d_5 (Conv2D)           (None, 4, 4, 128)         73856     
                                                                 
 leaky_re_lu_10 (LeakyReLU)  (None, 4, 4, 128)         0         
                                                                 
 flatten_1 (Flatten)         (None, 2048)             

In [ ]:
discriminator.compile(loss='binary_crossentropy', optimizer='adam',
                      metrics=['accuracy'])
discriminator.trainable = False

gan_model =Sequential()
gan_model.add(generator)
gan_model.add(discriminator)
gan_model.summary()
gan_model.compile(loss='binary_crossentropy', optimizer='adam')

real = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_4 (Sequential)   (None, 28, 28, 1)         1637121   
                                                                 
 sequential_5 (Sequential)   (None, 1)                 94721     
                                                                 
Total params: 1,731,842
Trainable params: 1,636,737
Non-trainable params: 95,105
_________________________________________________________________


In [ ]:
#그리고 포문
for epoch in range(epochs):
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    real_imgs = x_train[idx]

    z = np.random.normal(0, 1, (batch_size, noise))
    fake_imgs = generator.predict(z)

    d_hist_real = discriminator.train_on_batch(real_imgs, real)
    d_hist_fake = discriminator.train_on_batch(fake_imgs, fake)

    d_loss, d_acc = np.add(d_hist_fake, d_hist_real) * 0.5



    if epoch % 2 == 0:
        z = np.random.normal(0, 1, (batch_size, noise))
        gan_hist = gan_model.train_on_batch(z, real)

    if epoch % sample_interval == 0:
        print('%d, [D loss: %f, acc.: %.2f%%], [G loss: %f]'%(
                    epoch, d_loss, d_loss, gan_hist))
        row = col = 4
        z = np.random.normal(0, 1, (row * col, noise))
        fake_imgs = generator.predict(z)
        fake_imgs = 0.5 * fake_imgs + 0.5
        _, axs = plt.subplots(row, col, figsize=(5, 5), sharey=True, sharex=True)
        cont = 0
        for i in range(row):
            for j in range(col):
                axs[i, j].imshow(fake_imgs[cont, :, :, 0], cmap='gray')
                axs[i, j].axis('off')
                cont += 1
        path = os.path.join(OUT_DIR, 'img-{}'.format(epoch+1))
        plt.savefig(path)
        plt.close()


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
4/4 [==============================] - 0s 4ms/step
37100, [D loss: 0.198560, acc.: 0.20%], [G loss: 3.552632]
4/4 [==============================] - 0s 3ms/step
37200, [D loss: 0.169605, acc.: 0.17%], [G loss: 3.050279]
4/4 [==============================] - 0s 4ms/step
37300, [D loss: 0.179601, acc.: 0.18%], [G loss: 2.880195]
4/4 [==============================] - 0s 7ms/step
37400, [D loss: 0.181969, acc.: 0.18%], [G loss: 3.709342]
4/4 [==============================] - 0s 4ms/step
37500, [D loss: 0.186964, acc.: 0.19%], [G loss: 3.393340]
4/4 [==============================] - 0s 4ms/step
37600, [D loss: 0.262815, acc.: 0.26%], [G loss: 3.771752]
4/4 [==============================] - 0s 3ms/step
37700, [D loss: 0.208244, acc.: 0.21%], [G loss: 3.593039]
4/4 [==============================] - 0s 5ms/step
37800, [D loss: 0.128096, acc.: 0.13%], [G loss: 3.662854]
4/4 [==============================] - 0s 3ms/step
37900, [D loss: 0.159684, acc.: 0

In [ ]:

# lrelu = LeakyReLU(alpha=0.01) # 디폴트 값 : 0.3

# discriminator = Sequential()
# discriminator.add(Conv2D(32, kenel_size=3, strides=2, padding='same', input_shape=img_shape))
# discriminator.add(LeakyReLU(alpha=0.01))
# #맥스풀 생략 위에서 strides를 줬기 때문에
# discriminator.add(Conv2D(64, kenel_size=3, strides=2, padding='same'))  #레이어더쌓, 키워나가겠습니다
# discriminator.add(LeakyReLU(alpha=0.01))
# discriminator.add(Conv2D(128, kenel_size=3, strides=2, padding='same'))  #레이어더쌓, 키워나가겠습니다
# discriminator.add(LeakyReLU(alpha=0.01))
# discriminator.add(Flatten())# 덴스레이어 들어가야하니 flatten
# discriminator.add(Dense(1, activation='sigmoid'))
# discriminator.summary()




# discriminator.add(Flatten(input_shape=img_shape))
# discriminator.add(Dense(128, activation=lrelu))
# discriminator.add(Dense(1, activation='sigmoid'))
# discriminator.summary()
# discriminator.compile(loss='binary_crossentropy', optimizer='adam',
#                       metrics=['accuracy'])

# gan_model = Sequential()
# gan_model.add(generator)
# gan_model.add(discriminator)
# gan_model.summary()
# gan_model.compile(loss='binary_crossentropy', optimizer='adam')

# real = np.ones((batch_size, 1))     # np.ones : 1로 채워진 행렬로 만들어 주는 것
# #print(real)
# fake = np.zeros((batch_size, 1))
# #print(fake)

# for epoch in range(epochs):
#     idx = np.random.randint(0, x_train.shape[0], batch_size)    # 0-59999 랜덤으로 뽑아냄
#     real_imgs = x_train[idx]

#     z = np.random.normal(0, 1, (batch_size, noise))
#     fake_imgs = generator.predict(z)

#     d_hist_real = discriminator.train_on_batch(real_imgs, real)    # train_on_batch: 1회만 하고 그만둠
#     d_hist_fake = discriminator.train_on_batch(fake_imgs, fake)

#     d_loss, d_acc = np.add(d_hist_fake, d_hist_real) * 0.5  # 평균을 구함

#     discriminator.trainable = False

#     z = np.random.normal(0, 1, (batch_size, noise))
#     gan_hist = gan_model.train_on_batch(z, real) # 1이라고 답하게 학습

#     if epoch % sample_interval ==0:
#         print('%d, [D loss: %f, acc.: %.2f%%], [G loss: %f]'%(
#             epoch, d_loss, d_loss, gan_hist))
#         row = col = 4
#         z = np.random.normal(0, 1 ,(row*col, noise))
#         fake_imgs = generator.predict(z)
#         fake_imgs = 0.5 * fake_imgs + 0.5

#         _, axs = plt.subplots(row, col, figsize=(5, 5),sharey =True, sharex=True)
#         cont = 0
#         for i in range(row):
#             for j in range(col):
#                 axs[i, j].imshow(fake_imgs[cont, :, :, 0], cmap='gray')
#                 axs[i, j].axis('off')
#                 cont += 1
#         path = os.path.join(OUT_DIR, 'img-{}'.format(epoch + 1))
#         plt.savefig(path)
#         plt.close()
